# AmpliseQ Data Processing - Clean Generic Version

Generic pipeline for processing AmpliseQ results with configurable paths and sample filtering.

In [ ]:
import pandas as pd
from pathlib import Path
from typing import Optional

# ============================================
# CONFIGURATION
# ============================================

# Define your input/output directories here
INPUT_DIR = Path("/Users/tweber/Data/TREC/nf-core-ampliseq-results-downstream-test-090925")
OUTPUT_DIR = Path(
    "/Users/tweber/Gits/workspaces/depictio-workspace/depictio/depictio/api/v1/configs/ampliseq_dataset"
)

# Create output directory if it doesn't exist
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Optional: Define sample whitelist (None = process all samples)
SAMPLE_FILTER = None  # Example: ['SRR10070130', 'SRR10070131', 'SRR10070132']
SAMPLE_FILTER = [
    "sample_112506262",
    "sample_112506283",
    "sample_112506284",
    "sample_112506319",
    "sample_112506337",
    "sample_112506346",
    "sample_112506377",
    "sample_112506401",
    "sample_112506413",
    "sample_112506450",
    "sample_112506468",
    "sample_112506477",
    "sample_112506504",
    "sample_112506519",
    "sample_112506527",
    "sample_112506564",
    "sample_112506588",
    "sample_112506600",
    "sample_112506637",
    "sample_112506655",
    "sample_112506664",
    "sample_112506555",
    "sample_112506713",
    "sample_112506722",
    "sample_112506877",
    "sample_112506901",
    "sample_112506913",
    "sample_112494061",
    "sample_112494065",
    "sample_112494074",
    "sample_112494080",
    "sample_112494083",
    "sample_112494092",
    "sample_112494098",
    "sample_112494099",
    "sample_112494545",
    "sample_112494539",
    "sample_112494536",
    "sample_112494563",
    "sample_112494557",
    "sample_112494554",
    "sample_112494578",
    "sample_112494572",
    "sample_112494569",
    "sample_112507248",
    "sample_112507242",
    "sample_112494018",
    "sample_112499299",
    "sample_112499305",
    "sample_112499308",
    "sample_112499404",
    "sample_112499398",
    "sample_112499374",
    "sample_112494076",
]

# Condition column name - specify the metadata column to use for grouping samples
# Common values: 'condition' (new experiments), 'habitat' (old experiments)
# Set to 'auto' to auto-detect (tries 'condition' first, then 'habitat')
CONDITION_COLUMN_NAME = 'condition'  # Options: 'auto', 'condition', 'habitat', or any custom column name

## 1. Process Alpha Diversity (Faith PD)

Converts alpha rarefaction data from wide to long format.

In [ ]:
def process_faith_pd(
    input_dir: Path,
    output_dir: Path,
    sample_filter: Optional[list[str]] = None
) -> pd.DataFrame:
    """
    Process Faith PD alpha diversity data from wide to long format.
    
    Args:
        input_dir: Path to ampliseq results directory
        output_dir: Path to save output file
        sample_filter: Optional list of sample IDs to include
    
    Returns:
        DataFrame in long format with columns: sample, depth, iter, faith_pd
    """
    # Read input file
    input_file = input_dir / "qiime2/alpha-rarefaction/faith_pd.csv"
    df = pd.read_csv(input_file)
    
    # Apply sample filter if provided
    if sample_filter is not None:
        df = df[df['sample-id'].isin(sample_filter)]
    
    # Melt from wide to long format
    df_long = df.melt(
        id_vars=['sample-id'],
        var_name='iteration',
        value_name='faith_pd'
    )
    
    # Extract depth and iteration from column names
    df_long['depth'] = df_long['iteration'].str.extract(r'depth-(\d+)')[0].astype('Int64')
    df_long['iter'] = df_long['iteration'].str.extract(r'iter-(\d+)')[0].astype('Int64')
    
    # Clean up column names and select final columns
    df_modified = df_long.rename(columns={'sample-id': 'sample'})
    df_modified = df_modified[['sample', 'depth', 'iter', 'faith_pd']].dropna()
    
    # Save to output
    output_file = output_dir / "faith_pd_long.tsv"
    df_modified.to_csv(output_file, sep='\t', index=False)
    print(f"✓ Saved Faith PD data: {output_file}")
    print(f"  Shape: {df_modified.shape}")
    print(f"  Samples: {df_modified['sample'].nunique()}")
    
    return df_modified


# Execute function
df_faith = process_faith_pd(INPUT_DIR, OUTPUT_DIR, SAMPLE_FILTER)
df_faith.head()

## 2. Process Taxonomy Barplot Data

Converts taxonomy abundance data from wide to long format.

In [ ]:
def process_taxonomy_barplot(
    input_dir: Path,
    output_dir: Path,
    level: int = 2,
    sample_filter: Optional[list[str]] = None,
    condition_col_name: str = 'auto'
) -> pd.DataFrame:
    """
    Process taxonomy barplot data from wide to long format.
    
    Args:
        input_dir: Path to ampliseq results directory
        output_dir: Path to save output file
        level: Taxonomic level (default: 2 for Phylum)
        sample_filter: Optional list of sample IDs to include
        condition_col_name: Name of condition column ('auto' to auto-detect, or specific name)
    
    Returns:
        DataFrame in long format with columns: sample, taxonomy, count, condition, Kingdom, Phylum
    """
    # Read input file
    input_file = input_dir / f"qiime2/barplot/level-{level}.csv"
    df = pd.read_csv(input_file)
    
    # Apply sample filter if provided
    if sample_filter is not None:
        df = df[df['index'].isin(sample_filter)]
    
    # Identify columns: first is sample ID, detect metadata columns, rest are taxonomy
    sample_col = df.columns[0]  # 'index'
    
    # Auto-detect metadata columns at the end
    metadata_candidates = ['name', 'condition_binary', 'cycle', 'condition', 'bio_rep', 'habitat']
    metadata_cols = [col for col in df.columns if col in metadata_candidates]
    n_metadata = len(metadata_cols)
    
    # Get taxonomy columns (everything between sample ID and metadata)
    if n_metadata > 0:
        taxonomy_cols = df.columns[1:len(df.columns)-n_metadata]
    else:
        taxonomy_cols = df.columns[1:]
    
    print(f"  Detected {n_metadata} metadata columns: {metadata_cols}")
    print(f"  Detected {len(taxonomy_cols)} taxonomy columns")
    
    # Select taxonomy columns and sample ID
    df_samples = df[[sample_col] + list(taxonomy_cols)]
    
    # Melt to long format
    df_modified = df_samples.melt(
        id_vars=[sample_col],
        var_name='taxonomy',
        value_name='count'
    )
    
    original_count = len(df_modified)
    
    # Rename sample column
    df_modified = df_modified.rename(columns={sample_col: 'sample'})
    
    # Determine condition column to use
    if condition_col_name == 'auto':
        # Auto-detect: try 'condition' first, then 'habitat'
        condition_col = 'condition' if 'condition' in df.columns else ('habitat' if 'habitat' in df.columns else None)
    else:
        # Use specified column name
        condition_col = condition_col_name if condition_col_name in df.columns else None
    
    # Add condition information
    if condition_col:
        sample_to_condition = df.set_index(sample_col)[condition_col].to_dict()
        df_modified['condition'] = df_modified['sample'].map(sample_to_condition)
        print(f"  Using '{condition_col}' column for sample grouping")
    else:
        df_modified['condition'] = None
        print(f"  Warning: Condition column '{condition_col_name}' not found in data")
    
    # Parse taxonomy levels
    df_modified['Kingdom'] = df_modified['taxonomy'].str.split(';').str[0]
    df_modified['Phylum'] = df_modified['taxonomy'].str.split(';').str[1]
    
    # Filter out invalid taxonomy entries
    # Remove rows where:
    # - taxonomy is empty or just ";" or just whitespace
    # - Kingdom is empty or is a metadata column name
    # - Count is null or zero
    metadata_names = set(metadata_candidates)
    df_modified = df_modified[
        (df_modified['taxonomy'].notna()) &  # Not null
        (df_modified['taxonomy'].str.strip() != '') &  # Not empty after stripping
        (df_modified['taxonomy'].str.strip() != ';') &  # Not just separator
        (df_modified['Kingdom'].notna()) &  # Kingdom not null
        (df_modified['Kingdom'].str.strip() != '') &  # Kingdom not empty
        (~df_modified['Kingdom'].isin(metadata_names)) &  # Kingdom not a metadata column name
        (df_modified['count'].notna()) &  # Count not null
        (df_modified['count'] > 0)  # Count greater than zero
    ].copy()
    
    filtered_count = original_count - len(df_modified)
    if filtered_count > 0:
        print(f"  Filtered out {filtered_count} invalid/empty taxonomy entries")
    
    # Save to output
    output_file = output_dir / f"taxonomy_level{level}_long.tsv"
    df_modified.to_csv(output_file, sep='\t', index=False)
    print(f"✓ Saved taxonomy data: {output_file}")
    print(f"  Shape: {df_modified.shape}")
    print(f"  Samples: {df_modified['sample'].nunique()}")
    print(f"  Taxa: {df_modified['taxonomy'].nunique()}")
    
    return df_modified


# Execute function
df_taxonomy = process_taxonomy_barplot(INPUT_DIR, OUTPUT_DIR, level=2, sample_filter=SAMPLE_FILTER, condition_col_name=CONDITION_COLUMN_NAME)
df_taxonomy.head()

## 3. Process ANCOM Volcano Plot Data

Merges ANCOM differential abundance results with taxonomy information.

In [ ]:
def process_ancom_volcano(
    input_dir: Path,
    output_dir: Path,
    category: str = "habitat",
    level: str = "ASV",
    sample_filter: Optional[list[str]] = None
) -> pd.DataFrame:
    """
    Process ANCOM results and merge with taxonomy data for volcano plots.
    
    Args:
        input_dir: Path to ampliseq results directory
        output_dir: Path to save output file
        category: ANCOM category (e.g., 'habitat')
        level: Taxonomic level (e.g., 'ASV')
        sample_filter: Optional list of sample IDs to include (applied to taxonomy table)
    
    Returns:
        DataFrame with columns: id, taxonomy, Kingdom, Phylum, W, clr
    """
    # Read ANCOM results
    ancom_file = input_dir / f"qiime2/ancom/Category-{category}-{level}/data.tsv"
    df_ancom = pd.read_csv(ancom_file, sep='\t')
    
    # Read taxonomy/abundance table
    tax_file = input_dir / f"qiime2/rel_abundance_tables/rel-table-{level}_with-DADA2-tax.tsv"
    df_tax = pd.read_csv(tax_file, sep='\t')
    
    # Apply sample filter to taxonomy table if provided
    if sample_filter is not None:
        # Keep only columns that are in sample_filter (plus metadata columns)
        metadata_cols = ['ID', 'Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']
        cols_to_keep = [col for col in metadata_cols if col in df_tax.columns]
        cols_to_keep += [col for col in df_tax.columns if col in sample_filter]
        df_tax = df_tax[cols_to_keep]
    
    # Create combined taxonomy string
    df_tax['taxonomy'] = df_tax['Kingdom'] + ';' + df_tax['Phylum']
    
    # Merge ANCOM results with taxonomy
    df_modified = df_ancom.merge(
        df_tax[['ID', 'taxonomy', 'Kingdom', 'Phylum']],
        left_on='id',
        right_on='ID',
        how='left'
    )
    
    # Select and clean final columns
    df_modified = df_modified[['id', 'taxonomy', 'Kingdom', 'Phylum', 'W', 'clr']].dropna()
    
    # Save to output
    output_file = output_dir / f"ancom_{category}_{level}_volcano.tsv"
    df_modified.to_csv(output_file, sep='\t', index=False)
    print(f"✓ Saved ANCOM volcano data: {output_file}")
    print(f"  Shape: {df_modified.shape}")
    print(f"  ASVs: {df_modified['id'].nunique()}")
    print(f"  Phyla: {df_modified['Phylum'].nunique()}")
    
    return df_modified


# Execute function
df_ancom = process_ancom_volcano(INPUT_DIR, OUTPUT_DIR, category="habitat", level="ASV", sample_filter=SAMPLE_FILTER)
df_ancom.head()

## Summary

All processed files have been saved to the output directory:

In [ ]:
# List all output files
output_files = sorted(OUTPUT_DIR.glob("*.tsv"))
print(f"\nOutput files in {OUTPUT_DIR}:")
for f in output_files:
    size_mb = f.stat().st_size / (1024 * 1024)
    print(f"  - {f.name} ({size_mb:.2f} MB)")